In [2]:
#import 
import MDAnalysis as mda
from MDAnalysis.analysis import gnm
from MDAnalysis.analysis import rms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

In [12]:
evs = list()
unis = list()
refs = list()
nmas = list()
time = list(range(5001))       # fix time differences (scale by 10)

unis.append(mda.Universe("md_0_1_490.gro","md_0_1_490.xtc"))
refs.append(mda.Universe("gro_rmsdstructs/csa_acetone_noh.gro"))    # def ref unis
refs.append(mda.Universe("gro_rmsdstructs/csa_cyp_noh.gro"))    # def ref unis
refs.append(mda.Universe("gro_rmsdstructs/csa_fab_noh.gro"))    # def ref unis

In [13]:
#sel = 'not backbone and not type H and resnum 2:' 
sel = 'name CB and (resnum 1 or resnum 2 or resnum 3 or resnum 4 or resnum 5 or resnum 6 or resnum 7 or resnum 8 or resnum 9 or resnum 10)'
#sel = 'name CG or name CZ or name CN'
print('Target uni:\t'+str(len(u.select_atoms(sel).positions)))
print('Ref uni:\t'+str(len(r.select_atoms(sel).positions)))

Target uni:	9
Ref uni:	9


GNM

In [17]:
# sidechain atoms
for u in unis:
    for r in refs:
        nmas.append(gnm.GNMAnalysis(u, select=sel, cutoff=7.0))    # create,
        
for a in nmas:
    a.run()            #  run the analyses
    evs.append([res[1] for res in a.results])      # calculate eigenvalues
    #time.append([res[0] for res in nma[i].results])             # calculate times

In [20]:
evs = list()

histfig, histax = plt.subplots(nrows=len(unis)*len(refs), sharex=True, sharey=True)
for i in list(range(len(nmas))):
    histax[i].hist(evs[i])
    histax[i].set_ylabel('Frequency')
histax[0].set_xlabel('Eigenvalue')

IndexError: list index out of range

RMSD calculations

In [14]:
sc_rmss = list()
sc_rmsds = list()
sc_data = list()

for u in unis:
    for r in refs:
        sc_rmss.append(rms.rmsd(u.select_atoms(sel).positions, r.select_atoms(sel).positions, center=True, superposition=True))  # append the rms analysis to the given position in rmss
        sc_rmsds.append(rms.RMSD(u, r, select=sel, ref_frame=0))   # run RMSD analyses

for a in sc_rmsds:
    a.run()
    sc_data.append(pd.DataFrame(a.rmsd, columns=['Frame', 'Time (ps)', 'Residue']))

In [16]:
# prepare individual plots
ax = sc_data.plot(x='Frame', y=['Residue'], kind='line', figsize=(150,5), xlim=(-1,5001))
ax.set_ylabel('RMSD ($\AA$)')
ax.set_xlabel('Time (ps)')
#ax.xticks((1000*np.array(list(range(1,51)))))
#ax.figure.savefig("rmsd_vs_acetone.pdf", bbox_inches="tight")
ax2=ax.twinx()
ax2.plot(time, ev_cb, label='hello', color='red')    # fixed for now!!! how do I make this oscillate properly??? make a 2D array of times/eigenvals/labels?
ax.figure.savefig("cb_plt.pdf", bbox_inches="tight")

AttributeError: 'list' object has no attribute 'plot'